### 1. Entity에 따른 Special Token을 문장에 추가해보자!
### 2. Entity 유무를 나타내는 embedding layer를 추가해보자!

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import pandas as pd
import numpy as np
import re
import tarfile
import pickle as pickle
from tqdm import tqdm
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.model_selection import train_test_split

# Using KoELECTRA Model
from transformers import ElectraModel, ElectraTokenizer, ElectraForSequenceClassification

# Added by Me
import os
from tqdm.notebook import tqdm
from ohsuz.utils import *
from ohsuz.loss import *
from ohsuz.config import *
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-PCIE-32GB


### **1. Entity에 따른 Special Token을 문장에 추가해보자!**

In [2]:
dataset = pd.read_csv(os.path.join(train_dir, 'train.tsv'), delimiter='\t', header=None)

In [3]:
dataset.head()

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님


In [4]:
practice = dataset[1][0]

In [5]:
a1, a2, b1, b2 = dataset[3][0], dataset[4][0], dataset[6][0], dataset[7][0]

In [6]:
if a1 > b1: # b1 먼저
    new = practice[:b1] + "<E02>" + practice[b1:b2+1] + "</E02>" + practice[b2+1:a1] + "<E01>" + practice[a1:a2+1] + "</E01>" + practice[a2+1:]
    print(new)
else: # a1 먼저
    new = practice[:a1] + "<E01>" + practice[a1:a2+1] + "</E01>" + practice[a2+1:b1] + "<E02>" + practice[b1:b2+1] + "</E02>" + practice[b2+1:]
    print(new)

영국에서 사용되는 스포츠 유틸리티 <E02>자동차</E02>의 브랜드로는 <E01>랜드로버</E01>(Land Rover)와 지프(Jeep)가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다.


In [7]:
# 원하는 entity 네임을 지정할 수도 있게 하고 싶은데 그러면 파라미터가 너무 많아진다
def add_entity_tokens(sentence, a1, a2, b1, b2):
    new_sentence = None
    if a1 > b1: # b1 먼저
        new_sentence = sentence[:b1] + "[E02]" + sentence[b1:b2+1] + "[/E02]" + sentence[b2+1:a1] + "[E01]" + sentence[a1:a2+1] + "[/E01]" + sentence[a2+1:]
    else: # a1 먼저
        new_sentence = sentence[:a1] + "[E01]" + sentence[a1:a2+1] + "[/E01]" + sentence[a2+1:b1] + "[E02]" + sentence[b1:b2+1] + "[/E02]" + sentence[b2+1:]
    return new_sentence

In [8]:
text= add_entity_tokens(dataset[1][0], dataset[3][0], dataset[4][0], dataset[6][0], dataset[7][0])
print(text)

영국에서 사용되는 스포츠 유틸리티 [E02]자동차[/E02]의 브랜드로는 [E01]랜드로버[/E01](Land Rover)와 지프(Jeep)가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다.


### ***※※※ 새로운 토큰을 추가해줬을 때 유의할 점 ※※※***

> pretrained model은 기존 vocab size에 맞춰있기 때문에, token을 새로 추가하는 경우 vocab size를 조정해줘야 함

- tokenizer.add_special_tokens({'token_name':[name list]})
- model.resize_token_embeddings(tokenizer.vocab_size + added_token_num = len(tokenizer))

In [9]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [10]:
print(text, end='\n\n')
print(tokenizer.tokenize(text)) # special token이 일반 token처럼 분리된 것을 확인할 수 있음

영국에서 사용되는 스포츠 유틸리티 [E02]자동차[/E02]의 브랜드로는 [E01]랜드로버[/E01](Land Rover)와 지프(Jeep)가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다.

['영국', '##에', '##서', '사용', '##되', '##는', '스포츠', '유틸리티', '[', 'E', '##0', '##2', ']', '자동차', '[', '/', 'E', '##0', '##2', ']', '의', '브랜드', '##로', '##는', '[', 'E', '##0', '##1', ']', '랜드', '##로', '##버', '[', '/', 'E', '##0', '##1', ']', '(', 'La', '##nd', 'R', '##over', ')', '와', '지프', '(', 'Je', '##ep', ')', '가', '있', '##으며', ',', '이', '브랜드', '##들', '##은', '자동차', '##의', '종류', '##를', '일컫', '##는', '말', '##로', '사용', '##되', '##기', '##도', '한다', '.']


In [11]:
print(len(tokenizer))

35000


In [12]:
special_tokens_dict = {'additional_special_tokens': ['[E01]','[/E01]','[E02]','[/E02]']}
tokenizer.add_special_tokens(special_tokens_dict)

4

In [13]:
print(len(tokenizer))

35004


In [14]:
print(tokenizer.tokenize(text))

['영국', '##에', '##서', '사용', '##되', '##는', '스포츠', '유틸리티', '[E02]', '자동차', '[/E02]', '의', '브랜드', '##로', '##는', '[E01]', '랜드', '##로', '##버', '[/E01]', '(', 'La', '##nd', 'R', '##over', ')', '와', '지프', '(', 'Je', '##ep', ')', '가', '있', '##으며', ',', '이', '브랜드', '##들', '##은', '자동차', '##의', '종류', '##를', '일컫', '##는', '말', '##로', '사용', '##되', '##기', '##도', '한다', '.']


In [15]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=42)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'electra.embeddings.position_ids']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discri

In [16]:
print(model.get_input_embeddings())

Embedding(35000, 768, padding_idx=0)


In [17]:
model.resize_token_embeddings(len(tokenizer))
print(model.get_input_embeddings())

Embedding(35004, 768)


### **2. Entity 유무를 나타내는 embedding layer를 추가해보자!**

In [45]:
inputs = tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=190,
            pad_to_max_length=True,
            add_special_tokens=True
        )

In [46]:
inputs 

{'input_ids': tensor([[    2,  6642,  4073,  4129,  6267,  4479,  4034,  7347, 23927, 35002,
          6729, 35003,  3238,  6955,  4239,  4034, 35000, 10086,  4239,  4505,
         35001,    12, 22207,  7466,    54, 29841,    13,  3170, 25982,    12,
         25211,  9940,    13,  2010,  3249,  6460,    16,  3240,  6955,  4006,
          4112,  6729,  4234,  7890,  4110, 16461,  4034,  2633,  4239,  6267,
          4479,  4031,  4086,  6217,    18,     3,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [53]:
ids = inputs['input_ids'][0]
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

In [76]:
print(type(tokens))

<class 'list'>


- entity token의 id 값을 미리 구해놓고 그 사이에 있는 애들은 다 1, 아니면 다 0

In [73]:
flag = False
special_tokens = ['[E01]','[/E01]','[E02]','[/E02]']
is_entity_layer = []

for i, token in enumerate(tokens):
    if token in special_tokens:
        if flag == False:
            flag = True
        else:
            flag = False
    else:
        if flag == True:
            is_entity_layer.append(1)
            continue
    is_entity_layer.append(0)

is_entity_layer = torch.tensor(is_entity_layer)
is_entity_layer

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [74]:
is_entity_layer + inputs.attention_mask[0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])